# train

In [1]:
import yaml
import os

with open("./config.yaml", 'rb') as f:
    config = yaml.load(f)

<ipython-input-1-1499ab5789c7>:5: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  config = yaml.load(f)


In [2]:
OUTPUT_ROOT = config['IO_OPTION']['OUTPUT_ROOT']
os.makedirs(OUTPUT_ROOT, exist_ok=True)

## load library

In [3]:
# python default library
import os
import shutil
import datetime
import sys
import pickle

# general analysis tool-kit
import numpy as np
import pandas as pd
#from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import seaborn as sns

# pytorch
import torch
from torch import optim, nn
from torch.utils.tensorboard import SummaryWriter
import timm
import torch_optimizer
#from torchvision.models import wide_resnet50_2, resnet18
#from torchsummary import summary

# etc
import yaml
yaml.warnings({'YAMLLoadWarning': False})

# original library
import utils
import common as com
import pytorch_modeler as modeler
from pytorch_model import EfficientNet_b1 as Model
from pytorch_model import EWCLoss
#from pytorch_utils import filtered_load_model
#import models

import librosa
import IPython
import librosa.display

FISHER_DIR = config['IO_OPTION']['FISHER_ROOT']
fisher_path = f'{FISHER_DIR}/{machine_types[0]}_fisher.pkl'
fisher = pd.read_pickle(fisher_path)

## load config and set logger

In [4]:
log_folder = config['IO_OPTION']['OUTPUT_ROOT']+'/{0}.log'.format(datetime.date.today())
logger = com.setup_logger(log_folder, '00_train.py')

## Setting

In [5]:
# Setting seed
modeler.set_seed(42)

In [6]:
############################################################################
# Setting I/O path
############################################################################
# input dirs
INPUT_ROOT = config['IO_OPTION']['INPUT_ROOT']
dev_dir = INPUT_ROOT + "/dev_data"
add_dev_dir = INPUT_ROOT + "/add_dev_data"
# machine type
machine_types = os.listdir(dev_dir)
# output dirs
OUTPUT_ROOT = config['IO_OPTION']['OUTPUT_ROOT']
MODEL_DIR = config['IO_OPTION']['OUTPUT_ROOT'] + '/models'
PREMODEL_ROOT = config['IO_OPTION']['PREMODEL_ROOT']

TB_DIR = config['IO_OPTION']['OUTPUT_ROOT'] + '/tb'
OUT_FEATURE_DIR = OUTPUT_ROOT + '/extraction_features'
PRED_DIR = config['IO_OPTION']['OUTPUT_ROOT'] + '/pred'
#os.makedirs(OUTPUT_ROOT, exist_ok=True)
os.makedirs(MODEL_DIR, exist_ok=True)
os.makedirs(TB_DIR, exist_ok=True)
os.makedirs(OUT_FEATURE_DIR, exist_ok=True)
os.makedirs(PRED_DIR, exist_ok=True)

# copy config
shutil.copy('./config.yaml', OUTPUT_ROOT)

'/media/hiroki/HDD1TB/research/DCASE_experiments/domain_adaptation/exp4/PANDA/config.yaml'

In [7]:
torch.cuda.is_available()

True

## make path list

- __train_paths__ ([dict]): Each Machine Type
    - __ToyCar__ ([dict]): Each dataset
        - train ([list]) : paths
        - valid_source ([list]) : paths
        - valid_target ([list]) :paths
    - __ToyTrain__ ([dict]): Each dataset
        - train ([list]) : paths
        - valid_source ([list]) : paths
        - valid_target ([list]) : paths
    - ...

In [8]:
dev_paths = {}

for machine_type in machine_types:
    # dev train
    dev_train_paths = [f"{dev_dir}/{machine_type}/train/" + file for file in os.listdir(f"{dev_dir}/{machine_type}/train")]
    dev_train_paths = sorted(dev_train_paths)
    
    print(f'Machine {machine_type}', end=',')
    print(f'dev_train_size : {len(dev_train_paths)}', end=', ')
    dev_train_source_paths = [path for path in dev_train_paths if 'source' in path]
    print(f'source_size : {len(dev_train_source_paths)}', end=', ')
    dev_train_target_paths = [path for path in dev_train_paths if 'target' in path]
    print(f'target_size : {len(dev_train_target_paths)}', end=', ')
    # source mode
    dev_train_paths = dev_train_source_paths + dev_train_target_paths
    
    # add_dev train
    add_dev_paths = [f"{add_dev_dir}/{machine_type}/train/" + file for file in os.listdir(f"{add_dev_dir}/{machine_type}/train")]
    add_dev_paths = sorted(add_dev_paths)
    print(f'adddev_train_size : {len(dev_train_paths)}', end=', ')
    add_dev_source_paths = [path for path in add_dev_paths if 'source' in path]
    print(f'source_size : {len(add_dev_source_paths)}', end=', ')
    add_dev_target_paths = [path for path in add_dev_paths if 'target' in path]
    print(f'target_size : {len(add_dev_target_paths)}')
    # source mode
    add_dev_paths = add_dev_source_paths + add_dev_target_paths
    print('==============================================================')
    
    # dev_source valid
    dev_source_paths = [f"{dev_dir}/{machine_type}/source_test/" + file for file in os.listdir(f"{dev_dir}/{machine_type}/source_test")]
    dev_source_paths = sorted(dev_source_paths)
    
    # dev_target valid
    dev_target_paths = [f"{dev_dir}/{machine_type}/target_test/" + file for file in os.listdir(f"{dev_dir}/{machine_type}/target_test")]
    dev_target_paths = sorted(dev_target_paths)
    
    # bundle
    dev_paths[machine_type] = {}
    dev_paths[machine_type]['train'] = dev_train_paths + add_dev_paths
    dev_paths[machine_type]['valid_source'] = dev_source_paths
    dev_paths[machine_type]['valid_target'] = dev_target_paths

Machine fan,dev_train_size : 3009, source_size : 3000, target_size : 9, adddev_train_size : 3009, source_size : 3000, target_size : 9
Machine gearbox,dev_train_size : 3026, source_size : 3017, target_size : 9, adddev_train_size : 3026, source_size : 3105, target_size : 9
Machine pump,dev_train_size : 3009, source_size : 3000, target_size : 9, adddev_train_size : 3009, source_size : 3000, target_size : 9
Machine slider,dev_train_size : 3009, source_size : 3000, target_size : 9, adddev_train_size : 3009, source_size : 3000, target_size : 9
Machine ToyCar,dev_train_size : 3009, source_size : 3000, target_size : 9, adddev_train_size : 3009, source_size : 3000, target_size : 9
Machine ToyTrain,dev_train_size : 3009, source_size : 3000, target_size : 9, adddev_train_size : 3009, source_size : 3000, target_size : 9
Machine valve,dev_train_size : 3009, source_size : 3000, target_size : 9, adddev_train_size : 3009, source_size : 3000, target_size : 9


In [9]:
machine_types

['fan', 'gearbox', 'pump', 'slider', 'ToyCar', 'ToyTrain', 'valve']

In [10]:
from copy import deepcopy

In [11]:
#############################################################################
# run
#############################################################################
def run(machine_type, dev_paths):
    com.tic()
    os.makedirs(TB_DIR+'/'+machine_type, exist_ok=True)
    tb_log_dir = TB_DIR + '/' + machine_type
    writer = SummaryWriter(log_dir = tb_log_dir)
    
    logger.info('TARGET MACHINE_TYPE: {0}'.format(machine_type))
    logger.info('MAKE DATA_LOADER')
    # dev_train_paths
    dataloaders_dict = modeler.make_dataloader(dev_paths, machine_type)
    # parameter setting
    logger.info('TRAINING')
    model = Model(num_center = 1)
    # load weight
    premodel_path = f'{PREMODEL_ROOT}/{machine_type}_model.pkl'
    pretrained_dict = torch.load(premodel_path)
    model.load_state_dict(pretrained_dict, strict=False)
    # load fisher
    FISHER_DIR = config['IO_OPTION']['FISHER_ROOT']
    fisher_path = f'{FISHER_DIR}/{machine_type}_fisher.pkl'
    fisher = pd.read_pickle(fisher_path)
    # frozen_model
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    frozen_model = deepcopy(model).to(device)
    frozen_model.eval()
    utils.freeze_model(frozen_model)
    # set ewc loss
    ewc_criterion = EWCLoss(frozen_model, fisher)
    
    # partially freeze
    utils.freeze_parameters(model)
    
    optimizer = torch.optim.Adam(model.parameters(),lr=1e-3)
    output_dict, model, pred_df = modeler.run_training(model, dataloaders_dict, writer, optimizer, ewc_criterion)
    
    # output
    feature_out_path = f'{OUT_FEATURE_DIR}/{machine_type}_features.pkl'
    model_out_path = f'{MODEL_DIR}/{machine_type}_model.pkl'
    pred_out_path = f'{PRED_DIR}/{machine_type}_pred.csv'
    # save
    pd.to_pickle(output_dict, feature_out_path)
    torch.save(model.state_dict(), model_out_path)
    pred_df.to_csv(pred_out_path)
    logger.info(f'SAVE SUCCESS : {model_out_path}')
    
    com.toc()

# run

In [ ]:
for machine_type in machine_types:
    run(machine_type, dev_paths)

2021-09-28 16:20:45,366 - 00_train.py - INFO - TARGET MACHINE_TYPE: fan
2021-09-28 16:20:45,367 - 00_train.py - INFO - MAKE DATA_LOADER
2021-09-28 16:20:45,371 - 00_train.py - INFO - TRAINING
  0%|          | 0/24 [00:00<?, ?it/s]

use: cuda:0


100%|██████████| 3/3 [00:02<00:00,  1.36it/s]
2021-09-28 16:21:10,131 - pytorch_modeler.py - INFO - epoch:1/10, tr_loss:1.785526, src_loss:1.740317, src_mean_auc:0.685967, tgt_loss:1.560801, tgt_mean_auc:0.658700,


,AUC,pAUC
Source_0,0.550900,0.501579
Source_1,0.815400,0.789474
Source_2,0.691600,0.580526
Target_0,0.679500,0.506316
Target_1,0.610300,0.501053
Target_2,0.686300,0.528421
mean,0.672333,0.567895
h_mean,0.662630,0.553188


100%|██████████| 3/3 [00:02<00:00,  1.39it/s]
2021-09-28 16:21:29,211 - pytorch_modeler.py - INFO - epoch:2/10, tr_loss:1.747646, src_loss:1.696495, src_mean_auc:0.664133, tgt_loss:1.523682, tgt_mean_auc:0.662867,


...

100%|██████████| 3/3 [00:02<00:00,  1.31it/s]
2021-09-28 16:25:46,262 - pytorch_modeler.py - INFO - epoch:5/10, tr_loss:1.172392, src_loss:1.083550, src_mean_auc:0.664323, tgt_loss:1.086029, tgt_mean_auc:0.636657,


,AUC,pAUC
Source_0,0.590085,0.540775
Source_1,0.733196,0.529276
Source_2,0.669686,0.559073
Target_0,0.617091,0.550337
Target_1,0.681927,0.512039
Target_2,0.610952,0.532373
mean,0.650490,0.537312
h_mean,0.646864,0.536880


100%|██████████| 3/3 [00:02<00:00,  1.36it/s]
2021-09-28 16:26:05,381 - pytorch_modeler.py - INFO - epoch:6/10, tr_loss:1.141228, src_loss:1.051141, src_mean_auc:0.670291, tgt_loss:1.054280, tgt_mean_auc:0.641516,


,AUC,pAUC
Source_0,0.602247,0.535762
Source_1,0.746656,0.559599
Source_2,0.661970,0.546996
Target_0,0.634549,0.565465
Target_1,0.680556,0.526749
Target_2,0.609444,0.525480
mean,0.655904,0.543342
h_mean,0.652419,0.542909


  0%|          | 0/24 [00:00<?, ?it/s]

In [ ]:
#run(machine_types[1], dev_paths)

In [ ]:
feature_out_path = f'{OUT_FEATURE_DIR}/{machine_types[0]}_features.pkl'
tst = pd.read_pickle(feature_out_path)

In [ ]:
tst['val_tgt']

In [ ]:
plt.figure(figsize=(10,10))
plt.imshow(tst['val_src']['feature'].astype(float), aspect='auto')

In [ ]:
tst['val_src']['feature'].shape

In [ ]:
plt.figure(figsize=(10,10))
plt.imshow(tst['val_tgt']['feature'].astype(float), aspect='auto')